# Import Library

In [73]:
import requests
from gensim.corpora import Dictionary
from gensim.matutils import sparse2full
from textacy.preprocess import preprocess_text
from gensim.models.tfidfmodel import TfidfModel
from sklearn.feature_extraction.text import TfidfVectorizer

In [74]:
def xrange(x):
 
    return iter(range(x))

# Query MongoDB

In [1]:
import pymongo

In [2]:
# Load data dari Database MongoDB 
myClient = pymongo.MongoClient("mongodb://localhost:27017")
myDatabase = myClient.iStorage
myCollection = myDatabase.iDataset

In [3]:
def getQuery(category=None):
    data = []
    iQuery = myCollection.find(
        {"category": "{}".format(category)}, 
        {
            "title": 1,
            "cleanContent": 1
        }).limit(200)
    
    for query in iQuery:
        data.append(query)
        
    return data

In [4]:
iData = []

In [5]:
iData = getQuery('news')

In [6]:
paragraph = []
for data in iData:
    paragraph.append(data['cleanContent'].split("\n\n"))

In [7]:
paragraph

[['hakim binsar gultom berjalan kaki berlari sekretariat panitia seleksi calon pimpinan komisi pemberantasan korupsi pansel capim kpk gedung kementerian sekretariat negara jakarta kamis 4 juli 2019 16 15 wib hakim dikenal mengadili kopi sianida terdakwa jessica kumala wongso mengikuti seleksi capim kpk pendaftaran ditutup 16 00 wib menenteng berkas tangannya binsar masuk ruang sekretariat 10 menit hakim pengadilan provinsi bangka belitung ditemui wartawan binsar mendaftar melengkapi data doakan ya kalinya mengikuti seleksi capim kpk periode 2019 2023 hakim karier lolos pimpinan lembaga antirasuah menetapkan pelaku mudah penyidik penuntut penasihat hukum hakim kpk menarik keputusannya mendaftar capim kpk mencari pekerjaan gagal mengikuti seleksi calon hakim agung joobseaker warna pendaftaran capim kpk periode 2019 2023 ditutup pansel memeriksa syarat administrasi pendaftar menentukan lolos tahap bantuan lembaga penelusuran rekam jejak calon komisioner polri kejaksaan agung badan intelij

## Clean Text

In [9]:
import requests

In [10]:
stopwords = requests.get("https://raw.githubusercontent.com/masdevid/ID-Stopwords/master/id.stopwords.02.01.2016.txt").text.split("\n")

In [12]:
clean_paragraph_stopwords = []
for i in range(len(paragraph)):
    clean_paragraph_stopwords.append([word for word in paragraph[i].split() if word not in stopwords])

AttributeError: 'list' object has no attribute 'split'

In [25]:
paragraph_split = []
for i in range(len(paragraph)):
    paragraph_split.append(paragraph[i][0].split())

In [26]:
len(paragraph_split)

200

In [27]:
paragraph_split

[['hakim',
  'binsar',
  'gultom',
  'berjalan',
  'kaki',
  'berlari',
  'sekretariat',
  'panitia',
  'seleksi',
  'calon',
  'pimpinan',
  'komisi',
  'pemberantasan',
  'korupsi',
  'pansel',
  'capim',
  'kpk',
  'gedung',
  'kementerian',
  'sekretariat',
  'negara',
  'jakarta',
  'kamis',
  '4',
  'juli',
  '2019',
  '16',
  '15',
  'wib',
  'hakim',
  'dikenal',
  'mengadili',
  'kopi',
  'sianida',
  'terdakwa',
  'jessica',
  'kumala',
  'wongso',
  'mengikuti',
  'seleksi',
  'capim',
  'kpk',
  'pendaftaran',
  'ditutup',
  '16',
  '00',
  'wib',
  'menenteng',
  'berkas',
  'tangannya',
  'binsar',
  'masuk',
  'ruang',
  'sekretariat',
  '10',
  'menit',
  'hakim',
  'pengadilan',
  'provinsi',
  'bangka',
  'belitung',
  'ditemui',
  'wartawan',
  'binsar',
  'mendaftar',
  'melengkapi',
  'data',
  'doakan',
  'ya',
  'kalinya',
  'mengikuti',
  'seleksi',
  'capim',
  'kpk',
  'periode',
  '2019',
  '2023',
  'hakim',
  'karier',
  'lolos',
  'pimpinan',
  'lembaga',


# Gensim TF-IDF Model

In [29]:
from gensim.corpora import Dictionary
from gensim.matutils import sparse2full
from textacy.preprocess import preprocess_text
from gensim.models.tfidfmodel import TfidfModel
from sklearn.feature_extraction.text import TfidfVectorizer

In [30]:
docs_dict = Dictionary(paragraph_split)
docs_dict.filter_extremes(no_below=10, no_above=0.5)
docs_dict.compactify()

In [31]:
import numpy as np

docs_corpus = [docs_dict.doc2bow(doc) for doc in paragraph_split]
model_tfidf = TfidfModel(docs_corpus, id2word=docs_dict)
docs_tfidf  = model_tfidf[docs_corpus]
docs_vecs   = np.vstack([sparse2full(c, len(docs_dict)) for c in docs_tfidf])

In [32]:
docs_vecs.shape

(200, 443)

# Annoy

In [34]:
def xrange(x):
 
    return iter(range(x))

In [35]:
from annoy import AnnoyIndex
import random

f = docs_vecs.shape[1]
t = AnnoyIndex(f)  # Length of item vector that will be indexed
for i in xrange(len(paragraph_split)):
    v = docs_vecs[i]
    t.add_item(i, v)

t.build(100) # 10 trees
t.save('test.ann')

# ...

u = AnnoyIndex(f)
u.load('test.ann') # super fast, will just mmap the file
print(u.get_nns_by_item(1, 10, include_distances=True)) # will find the 1000 nearest neighbors
temp = u.get_nns_by_item(1, 10, include_distances=True)

([1, 27, 18, 33, 143, 109, 158, 180, 5, 10], [0.0, 1.0113338232040405, 1.1163976192474365, 1.1358922719955444, 1.1472249031066895, 1.175337791442871, 1.178446650505066, 1.2353997230529785, 1.2419817447662354, 1.24448561668396])


In [37]:
temp[1]

[0.0,
 1.0113338232040405,
 1.1163976192474365,
 1.1358922719955444,
 1.1472249031066895,
 1.175337791442871,
 1.178446650505066,
 1.2353997230529785,
 1.2419817447662354,
 1.24448561668396]

In [38]:
paragraph

[['hakim binsar gultom berjalan kaki berlari sekretariat panitia seleksi calon pimpinan komisi pemberantasan korupsi pansel capim kpk gedung kementerian sekretariat negara jakarta kamis 4 juli 2019 16 15 wib hakim dikenal mengadili kopi sianida terdakwa jessica kumala wongso mengikuti seleksi capim kpk pendaftaran ditutup 16 00 wib menenteng berkas tangannya binsar masuk ruang sekretariat 10 menit hakim pengadilan provinsi bangka belitung ditemui wartawan binsar mendaftar melengkapi data doakan ya kalinya mengikuti seleksi capim kpk periode 2019 2023 hakim karier lolos pimpinan lembaga antirasuah menetapkan pelaku mudah penyidik penuntut penasihat hukum hakim kpk menarik keputusannya mendaftar capim kpk mencari pekerjaan gagal mengikuti seleksi calon hakim agung joobseaker warna pendaftaran capim kpk periode 2019 2023 ditutup pansel memeriksa syarat administrasi pendaftar menentukan lolos tahap bantuan lembaga penelusuran rekam jejak calon komisioner polri kejaksaan agung badan intelij

In [39]:
final_data = []
for i in range(len(paragraph)):
    u = AnnoyIndex(f)
    u.load('test.ann') # super fast, will just mmap the file
    print(u.get_nns_by_item(i, 10, include_distances=True)) # will find the 1000 nearest neighbors
    temp = u.get_nns_by_item(i, 10, include_distances=True)
    
    final_data.append([temp[0][i] for i in range(len(temp[1])) if temp[1][i] < 1])

([0, 182, 28, 150, 134, 122, 20, 94, 179, 3], [0.0, 1.0109796524047852, 1.0153762102127075, 1.0350408554077148, 1.058906078338623, 1.0721592903137207, 1.1194965839385986, 1.1262321472167969, 1.1411983966827393, 1.142816185951233])
([1, 27, 18, 33, 143, 109, 158, 180, 5, 10], [0.0, 1.0113338232040405, 1.1163976192474365, 1.1358922719955444, 1.1472249031066895, 1.175337791442871, 1.178446650505066, 1.2353997230529785, 1.2419817447662354, 1.24448561668396])
([2, 17, 161, 154, 31, 8, 76, 45, 55, 116], [0.0, 1.0288554430007935, 1.0771560668945312, 1.0986806154251099, 1.1046286821365356, 1.115015983581543, 1.116961121559143, 1.12734854221344, 1.1366729736328125, 1.1441099643707275])
([3, 94, 182, 0, 177, 163, 28, 59, 38, 20], [0.0, 0.9384856820106506, 1.02741539478302, 1.142816185951233, 1.1554694175720215, 1.191598653793335, 1.2006185054779053, 1.2095379829406738, 1.2160886526107788, 1.2287808656692505])
([4, 114, 197, 24, 129, 108, 150, 105, 130, 157], [0.0, 0.8104327917098999, 1.234696149

In [140]:
# final_data = []
# final_data.append([temp[0][i] for i in range(len(temp[1])) if temp[1][i] < 1])

In [40]:
final_data[0]

[0]

In [41]:
for fd in final_data[0]:
    print(paragraph[fd])
    print()

['hakim binsar gultom berjalan kaki berlari sekretariat panitia seleksi calon pimpinan komisi pemberantasan korupsi pansel capim kpk gedung kementerian sekretariat negara jakarta kamis 4 juli 2019 16 15 wib hakim dikenal mengadili kopi sianida terdakwa jessica kumala wongso mengikuti seleksi capim kpk pendaftaran ditutup 16 00 wib menenteng berkas tangannya binsar masuk ruang sekretariat 10 menit hakim pengadilan provinsi bangka belitung ditemui wartawan binsar mendaftar melengkapi data doakan ya kalinya mengikuti seleksi capim kpk periode 2019 2023 hakim karier lolos pimpinan lembaga antirasuah menetapkan pelaku mudah penyidik penuntut penasihat hukum hakim kpk menarik keputusannya mendaftar capim kpk mencari pekerjaan gagal mengikuti seleksi calon hakim agung joobseaker warna pendaftaran capim kpk periode 2019 2023 ditutup pansel memeriksa syarat administrasi pendaftar menentukan lolos tahap bantuan lembaga penelusuran rekam jejak calon komisioner polri kejaksaan agung badan intelije

# Testing

In [47]:
paragraph[1]

['penyidik bareskrim polri menjadwalkan pemeriksaan perdana mantan kepala staf komando cadangan strategis angkatan darat mayjen tni purn kivlan zen tersangka rabu 29 mei 2019 kivlan resmi ditetapkan tersangka dugaan penyebaran berita bohong makar polisi 27 mei 2019 rencananya besok 29 mei 10 00 wib pengacaranya bilang hadir kepala biro penerangan masyarakat mabes polri brigadir jenderal dedi prasetyo kantor kementerian koordinator bidang politik hukum keamanan jakarta pusat selasa 28 mei 2018 kivlan dilaporkan bernama jalaludin serang banten nomor laporan lp b 0442 v 2019 bareskrim perkara dilaporkan tindak pidana penyebaran berita bohong hoaks uu nomor 1 1946 kuhp pasal 14 pasal 15 keamanan negara makar uu nomor 1 1946 kuhp pasal 107 juncto pasal 87 pasal 163 bis juncto pasal 107 lantaran kivlan dicekal negeri ditjen imigrasi kementerian hukum ham pencekalan diserahkan kivlan bandara soekarno hatta tangerang banten 10 mei 2019 kepolisian kivlan pesawat brunei darussalam kota batam seh

In [48]:
paragraph[27]

['direktur kantor hukum ham lokataru haris azhar aparat hukum hati hati pasal makar menjerat aktivis menyuarakan people power harris permasalahan makar politis menyeret pendukung prabowo subianto kategori makar kuhp pasal definisi hati hati digunakannya unsurnya dipenuhi haris dihubungi tempo rabu 15 mei 2019 orang dilaporkan polisi dugaan makar mantan kepala staf komando cadangan strategis angkatan darat kostrad mayor jenderal tni purn kivlan zen politikus senior partai gerindra permadi aktivis lieus sungkharisma pengacara eggi sudjana eggi sudjana ditetapkan tersangka dugaan pelaku makar haris menyebut menerka nerka polisi memiliki bukti menjerat eggi pegang fakta faktanya bacot ya kuat harris harris pasal makar sebatas omongan polisi membuktikan kekuatan makar disiapkan ditempuh pakar hukum pidana universitas trisakti abdul fickar hadjar makar konteks hukum diatur pasal 104 106 107 kitab undang undang hukum pidana intinya diartikan membunuh presiden wakil presiden memisahkan wilayah

In [118]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [119]:
tfidf = TfidfVectorizer()

In [124]:
doc1 = tfidf.fit_transform(clean_paragraph)

In [125]:
doc1.shape

(1000, 5279)

In [68]:
from annoy import AnnoyIndex
import random

f = doc1.shape[1]
t = AnnoyIndex(f)  # Length of item vector that will be indexed
for i in xrange(len(clean_paragraph)):
    v = doc1.toarray()[i]
    t.add_item(i, v)

t.build(100) # 10 trees
t.save('test.ann')

# ...

u = AnnoyIndex(f)
u.load('test.ann') # super fast, will just mmap the file
print(u.get_nns_by_item(0, 10, include_distances=True)) # will find the 1000 nearest neighbors


NameError: name 'doc1' is not defined

In [135]:
paragraph[0]

"Koalisi Indonesia Kerja sedang menyusun visi dan misi pasangan Joko Widodo-Ma'ruf Amin. Sekretaris Jenderal Partai NasDem Johnny G Plate mengatakan koalisi mengupayakan visi-misi yang konkret dan masuk akal."

In [143]:
paragraph[99]

"L. Koalisi Indonesia Kerja sudah ajukan tiga nama kepada capres petahana Joko Widodo untuk dipilih sebagai Ketua Tim Kampanye Nasional pasangan Joko Widodo-Ma'ruf Amin."

# Test Sentiment

In [160]:
negative = requests.get("https://raw.githubusercontent.com/masdevid/ID-OpinionWords/master/negative.txt").text.split("\n")

In [162]:
positive = requests.get("https://raw.githubusercontent.com/masdevid/ID-OpinionWords/master/positive.txt").text.split("\n")

In [200]:
import re
from collections import OrderedDict
import numpy as np

In [204]:
class sentistrength:
    def __init__(self, config=dict()):
        self.negasi = [line.replace('\n','') for line in open("negatingword.txt").read().splitlines()]
        self.tanya = [line.replace('\n','') for line in open("questionword.txt").read().splitlines()]
        #create sentiment words dictionary
        self.sentiwords_txt = [line.replace('\n','').split(":") for line in open("sentiwords_id.txt").read().splitlines()]
        self.sentiwords_dict = OrderedDict()
        for term in self.sentiwords_txt:
            self.sentiwords_dict[term[0]] = int(term[1])
        #create emoticon dictionary
        self.emoticon_txt = [line.replace('\n','').split(" | ") for line in open("emoticon_id.txt").read().splitlines()]
        self.emoticon_dict = OrderedDict()
        for term in self.emoticon_txt:
            self.emoticon_dict[term[0]] = int(term[1])
        #create idioms dictionary
        self.idioms_txt = [line.replace('\n','').split(":") for line in open("idioms_id.txt").read().splitlines()]
        self.idioms_dict = OrderedDict()
        for term in self.idioms_txt:
            self.idioms_dict[term[0]] = int(term[1])
        #create boosterwords dictionary
        self.boosterwords_txt = [line.replace('\n','').split(":") for line in open("boosterwords_id.txt").read().splitlines()]
        self.boosterwords_dict = OrderedDict()
        for term in self.boosterwords_txt:
            self.boosterwords_dict[term[0]] = int(term[1])
        self.negation_conf = config["negation"]
        self.booster_conf = config["booster"]
        self.ungkapan_conf = config["ungkapan"]
        self.consecutive_conf = config["consecutive"]
        self.repeated_conf = config["repeated"]
        self.emoticon_conf = config["emoticon"]
        self.question_conf = config["question"]
        self.exclamation_conf = config["exclamation"]
        self.punctuation_conf = config["punctuation"]
        self.mean_conf = False

    def senti(self,term):
        try:
            return self.sentiwords_dict[term]
        except:
            return 0

    def emosikon(self,term):
        try:
            return self.emoticon_dict[term]
        except:
            return 0

    def ungkapan(self,term):
        try:
            return self.idioms_dict[term]
        except:
            return 0

    def booster(self, term):
        try:
            return self.boosterwords_dict[term]
        except:
            return 0

    def cek_negationword(self, prev_term, prev_term2):
        #jika kata sebelumnya (index-1) adalah kata negasi, negasikan nilai -+nya
        if prev_term in self.negasi or prev_term2+" "+prev_term in self.negasi:
            # print prev_term
            self.score = -abs(self.score) if self.score>0 else abs(self.score)

    def cek_boosterword(self,term):
        booster_score = self.booster(term)
        if booster_score !=0 and self.score>0: self.score += booster_score
        if booster_score !=0 and self.score<0: self.score -= booster_score

    def cek_consecutive_term(self, prev_term):
        if self.prev_score>0 and self.score >=3: self.score+=1
        if self.prev_score<0 and self.score <=-3: self.score-=1

    def cek_ungkapan(self, bigram,trigram, i):
        bigram = ' '.join(bigram)
        trigram = ' '.join(trigram)
        ungkapan_score = self.ungkapan(bigram)
        if ungkapan_score==0:
            ungkapan_score = self.ungkapan(trigram)
        if ungkapan_score!=0:
            self.score = ungkapan_score
            self.prev_score = 0
            self.pre_max_pos[i-1] = 1
            self.pre_max_neg[i-1] = -1
            self.max_pos = self.pre_max_pos[i-2] #if len(self.pre_max_pos)>1 else 1
            self.max_neg = self.pre_max_neg[i-2] #if len(self.pre_max_neg)>1 else -1
            self.sentence_score[i-1] = re.sub(r'\[\d\]','',self.sentence_score[i-1])

    def cek_repeated_punctuation(self, next_term):
        if re.search(r'!{2,}',next_term) and self.score >=3: self.score+=1
        if re.search(r'!{2,}',next_term) and self.score <=-3: self.score-=1

    def remove_extra_repeated_char(self, term):
        return re.sub(r'([A-Za-z])\1{2,}',r'\1',term)
    def plural_to_singular(self, term):
        return re.sub(r'([A-Za-z]+)\-\1', r'\1',term)
    def classify(self):
        result = "neutral"
        try:
            if self.mean_conf:
                mean_p = np.mean(self.mean_pos)
                mean_n = np.mean(self.mean_neg)
                print (mean_p, mean_n)
                if mean_p > mean_n:
                    result = "positive"
                elif mean_p < mean_n and not self.is_tanya:
                    result = "negative"
                elif mean_p < mean_n and self.is_tanya:
                    result = "neutral"
            else:
                if abs(self.sentences_max_pos) > abs(self.sentences_max_neg):
                    result = "positive"
                elif abs(self.sentences_max_pos) < abs(self.sentences_max_neg):
                    result = "negative"
                elif abs(self.sentences_max_pos) == abs(self.sentences_max_neg):
                    result = "neutral"
        except:
            print ("error ",self.sentences_max_pos, self.sentences_max_neg)
        return result
    def cek_neutral_term(self,terms,i):
        if terms[i-1] in self.neutral_term or terms[i+1] in self.neutral_term: self.score=1

    def main(self,sentence):
        self.neutral_term = ['jika','kalau']
        sentences = sentence.split('.')
        self.sentences_max_neg = -1
        self.sentences_max_pos = 1
        self.sentences_score = []
        self.sentences_text = []
        for sentence in sentences:
            self.max_neg = -1
            self.max_pos = 1
            self.mean_neg = [1]
            self.mean_pos = [1]
            self.sentence_score=[]
            terms = sentence.split()
            # terms = re.split(r'[\s,.]',sentence)
            terms_length = len(terms)
            self.is_tanya = False
            self.sentence_text = ''
            # print self.max_pos, self.max_neg
            #SEMUA KALIMAT YANG MEMILIKI TANDA SERU MEMILIKI +ve minimal 2
            if self.exclamation_conf and re.search('!',sentence): self.max_pos = 2
            self.prev_score = 0
            self.pre_max_pos = []
            self.pre_max_neg = []
            for i,term in enumerate(terms):
                # repeated_term = ''
                is_extra_char = False
                plural = ''
                self.score = 0
                # if re.search(r'[A-Za-z\-.]+',term):
                # print term
                if re.search(r'([A-Za-z])\1{3,}',term):
                    is_extra_char = True
                    # repeated_term =term
                term = self.remove_extra_repeated_char(term)
                if re.search(r'([A-Za-z]+)\-\1',term):
                    plural = term
                    term = self.plural_to_singular(term)
                #GET SENTI SCORE#
                self.score = self.senti(term)
                # print "senti score",term, self.score

                #NEGATION HANDLER#
                if self.negation_conf and self.score !=0 and i>0:self.cek_negationword(terms[i-1],terms[i-2])
                # print  "negation score",term, self.score

                #BOOSTERWORD HANDLER#
                if self.booster_conf and self.score !=0 and i>0 and i<=(terms_length-1):self.cek_boosterword(terms[i-1])
                if self.booster_conf and self.score !=0 and i>=0 and i<(terms_length-1):self.cek_boosterword(terms[i+1])
                # print  "booster score",term, self.score

                #IDIOM/UNGKAPAN HANDLER#
                if self.ungkapan_conf and i>0 and i<=(terms_length-1):self.cek_ungkapan([terms[i-1],term],[terms[i-2],terms[i-1],term],i)
                # if self.ungkapan_conf and i>=0 and i<(terms_length-1):self.cek_ungkapan([term,terms[i+1]])
                # print  "idiom score",term, self.score

                #CONSECUTIVE SENTIMENT WORD#
                if self.consecutive_conf and i>0 and i<=(terms_length-1) and self.score !=0:self.cek_consecutive_term(terms[i-1])
                # print  "consecutive score",term, self.score

                #+1 SENTI SCORE IF REPEATED CHAR ON POSITIVE/NEGATIVE +2 IF NEUTRAL TERM
                if self.repeated_conf and is_extra_char==True and self.score>0: self.score+=1
                if self.repeated_conf and is_extra_char==True and self.score<0: self.score-=1
                if self.repeated_conf and is_extra_char==True and self.score==0: self.score=2
                # print  "repeat char score", term, self.score
                if self.punctuation_conf and i>=0 and i<(terms_length-1): self.cek_repeated_punctuation(terms[i+1])
                # CEK APAKAH TERDAPAT KATA TANYA
                if self.question_conf and (term in self.tanya or re.search(r'\?',term)):self.is_tanya = True
                # CEK neutral term
                if self.score!=0 and i>1 and i<(terms_length-2): self.cek_neutral_term(terms,i)
                # if self.score!=0 and i>0 and i<(terms_length-4): self.cek_neutral_term(terms,i)
                if self.emoticon_conf and self.score==0: self.score = self.emosikon(term)

                self.prev_score = self.score
                if self.mean_conf and self.score>0: self.mean_pos.append(self.score)
                if self.mean_conf and self.score<0: self.mean_neg.append(abs(self.score))
                #GET MAX SCORE +ve/-ve
                self.max_pos= self.score if self.score > self.max_pos else self.max_pos
                self.max_neg= self.score if self.score < self.max_neg else self.max_neg
                #insert score info current term
                self.pre_max_pos.append(self.max_pos)
                self.pre_max_neg.append(self.max_neg)
                # print self.pre_max_pos, self.pre_max_neg
                if plural !='': term = plural
                self.sentence_text += ' {}'.format(term)
                if self.score != 0:term = "{} [{}]".format(term, self.score)
                self.sentence_score.append(term)

            self.sentences_text.append(self.sentence_text)
            self.sentences_score.append(" ".join(self.sentence_score))
            if self.is_tanya:
                self.max_neg = -1
            self.sentences_max_pos = self.max_pos if self.max_pos > self.sentences_max_pos else self.sentences_max_pos
            self.sentences_max_neg = self.max_neg if self.max_neg < self.sentences_max_neg else self.sentences_max_neg
            # print self.sentences_max_pos, self.sentences_max_neg
        sentence_result = self.classify()
        # print self.sentences_text
        return {"classified_text":". ".join(self.sentences_score),"tweet_text":". ".join(self.sentences_text),"sentence_score":self.sentences_score,"max_positive":self.sentences_max_pos,"max_negative":self.sentences_max_neg,"kelas":sentence_result}
    


In [236]:
config = dict()
config["negation"] = True
config["booster"]  = True
config["ungkapan"]  = True
config["consecutive"]  = True
config["repeated"]  = True
config["emoticon"]  = True
config["question"]  = True
config["exclamation"]  = True
config["punctuation"]  = True
senti = sentistrength(config)
print (senti.main("agnezmo pintar dan cantik sekali tetapi lintah darat :)"))

{'classified_text': 'agnezmo pintar [4] dan cantik [6] sekali tetapi lintah darat [-4] :) [3]', 'tweet_text': ' agnezmo pintar dan cantik sekali tetapi lintah darat :)', 'sentence_score': ['agnezmo pintar [4] dan cantik [6] sekali tetapi lintah darat [-4] :) [3]'], 'max_positive': 6, 'max_negative': -4, 'kelas': 'positive'}


In [223]:
data_positive, data_negative, data_neutral = [], [], []
for i in range(len(clean_paragraph)):
    
    if senti.main(clean_paragraph[i])['kelas'] == 'neutral':
        data_neutral.append(senti.main(clean_paragraph[i]))
    elif senti.main(clean_paragraph[i])['kelas'] == 'positive':
        data_positive.append(senti.main(clean_paragraph[i]))
    else:
        data_negative.append(senti.main(clean_paragraph[i]))

In [225]:
len(data_neutral)

383

In [226]:
len(data_positive)

403

In [227]:
len(data_negative)

214

In [228]:
total = len(data_neutral) + len(data_positive) + len(data_negative)

In [229]:
total

1000

In [237]:
data_negative

[{'classified_text': 'presiden amerika serikat donald trump mencabut [-2] izin keamanan mantan kepala cia john brennan pekan ini',
  'tweet_text': ' presiden amerika serikat donald trump mencabut izin keamanan mantan kepala cia john brennan pekan ini',
  'sentence_score': ['presiden amerika serikat donald trump mencabut [-2] izin keamanan mantan kepala cia john brennan pekan ini'],
  'max_positive': 1,
  'max_negative': -2,
  'kelas': 'negative'},
 {'classified_text': 'turki siap [4] untuk membahas masalah [-4] yang sedang berlangsung dengan amerika serikat selama tidak ada ancaman [-5]',
  'tweet_text': ' turki siap untuk membahas masalah yang sedang berlangsung dengan amerika serikat selama tidak ada ancaman',
  'sentence_score': ['turki siap [4] untuk membahas masalah [-4] yang sedang berlangsung dengan amerika serikat selama tidak ada ancaman [-5]'],
  'max_positive': 4,
  'max_negative': -5,
  'kelas': 'negative'},
 {'classified_text': 'wali kota parepare taufan pawe mengimbau kep